<a href="https://colab.research.google.com/github/luqmanr/web-scraping/blob/master/collect-data-instagram/instagram-scraping-tyogo/instagram_crawler_v4_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!python3 -m pip install opencv-contrib-python>=4.1.0
!pip install stdiomask
!apt install chromium-chromedriver
!pip install selenium
!pip install cp
!sudo apt-get install python-yaml
!sudo yum install python-yaml
!apt-get update
!apt install python3-dev python3-pip
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

In [0]:
import getpass

login_id = input('Your IG Account Username: ')
password = getpass.getpass('Your IG Account Password: ')

Your IG Account Username: tyogotest1
Your IG Account Password: ··········


In [0]:
usernames_target = [
  "tyogotest2"
]

In [0]:
import os, time, errno, pickle
import re
import json
import csv
import yaml
import sys
import cp
import selenium
from selenium import webdriver
from urllib.request import urlopen
from optparse import OptionParser
from selenium import webdriver  
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys

# di fungsi-fungsi-nya itu tyo jelaskan:
  # kenapa tyo manggil fungsi ini, 
  # bentuk data yang harus dimasukkan seperti apa, 
  # bentuk data yang akan diperoleh seperti apa

## Fungsi login Instagram dengan username akun dan password yang sudah dimasukkan pada sel ketiga
def login_instagram():
    try:
      driver.get("https://www.instagram.com/accounts/login/")
      time.sleep(2)
      driver.find_element_by_name("username").send_keys(login_id)
      driver.find_element_by_name("password").send_keys(password)
      time.sleep(1)
      driver.find_element_by_class_name("L3NKy").click()
      return login_id
    except:
      sys.stderr.write("the Xpath of this element (login_instagram) has changed \n")
      time.sleep(3)
      driver.close()
      sys.exit()

## Mengambil informasi nama lengkap, jumlah post, dan jumlah following dari profil Instagram username yang dicrawl
def userProfile():
  try:
    followingButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[3]/a/span'
    following_button = driver.find_element_by_xpath(followingButtonXpath).text
    print("Total Following: " + following_button)
  except:
    following_button = " "
    print("Total Following: " + following_button)
  try:
    postButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[1]/span/span'
    post_button = driver.find_element_by_xpath(postButtonXpath).text
    print("Total posts: " + post_button)
  except:
    post_button = " "
    print("Total posts: " + post_button)
  try:
    fullNameXpath = '/html/body/div[1]/section/main/div/header/section/div[2]/h1'
    full_name = driver.find_element_by_xpath(fullNameXpath).text
    print("Full Name: " + full_name)
  except:
    full_name = " "
    print("Full Name: " + full_name)
  return full_name, post_button, following_button

## Mengambil nilah dari jumlah follower akun username yang dicrawl.
def followerButton():
  try:
    follButtonXpath = '/html/body/div[1]/section/main/div/header/section/ul/li[2]/a/span'      
  except KeyError:
    ## xpath alternatif, sebagai antisipasi apabila Instagram mengganti API nya.
    follButtonXpath = '/html/body/div[1]/section/main/div/ul/li[2]/span'
    print("followerButton Xpath alt 2")
  except:
    sys.stderr.write("the Xpath of this element (followerButton) has changed \n")
    pass
  finally:
    follower_button = driver.find_element_by_xpath(follButtonXpath)
    printfoll = follower_button.get_attribute('title')
  ## Split nilai dari jumlah followers apabila lebih dari 999, karena akan muncul koma bila nilainya ribuan, jutaan, dst.
  follValue = printfoll.split(',')
  # print(follValue[0])
  # print(follValue[1])
  ## Kumpulkan hasil dari nilai yang displit itu menjadi satu.
  try:
    follvalue0 = follValue[0]
    follvalue1 = follValue[1]
    follvalue2 = follValue[2]
    follvalue = str(follvalue0 + follvalue1 + follvalue2)
    print("Total followers: " + follvalue)
    follower_button.click()
    return follvalue
  except IndexError:
    follvalue0 = follValue[0]
    follvalue1 = follValue[1]
    follvalue = str(follvalue0 + follvalue1)
    print("Total followers: " + follvalue)
    follower_button.click()
    return follvalue
  except IndexError:
    follvalue0 = follValue[0]
    follvalue = str(follvalue0)
    follvalue = follvalue0
    print("Total followers: " + follvalue)
    follower_button.click()
    return follvalue
  except:
    sys.stderr.write("Followers index out of range \n")

## Mengambil username-username yang ada pada list follower akun yang usernamenya dicrawl
def follGet(indexOfFollower):
  follmax = 300
  while indexOfFollower <= follmax:
    str_indexOfFollower = str(indexOfFollower)
    try:
      follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_indexOfFollower + ']/div/div[1]/div[2]/div[1]/a'
    except KeyError:
      ## xpath alternatif, sebagai antisipasi apabila Instagram mengganti API nya
      follXpath = '/html/body/div[4]/div/div[2]/ul/div/li[' + str_indexOfFollower + ']/div/div[2]/div[1]/div/div/a'
    except:
        pass
        sys.stderr.write("the Xpath of this element (follGet) has changed \n")
    finally:
      follower_name = driver.find_element_by_xpath(follXpath)
      follvalue = (follower_name.text)
      time.sleep(0.2)
      return follvalue
    if n > follmax:
      break
    else:
      pass

'''def ID():
    urlmax = 300  #maximal userID to take (to avoid account block by Instagram)
    while counter <= urlmax:
        driver.switch_to.window(driver.window_handles[1])
        driver.get("https://www.instagram.com/"+USERstr+"/?__a=1")
        time.sleep(0.3)
        userID=driver.find_element_by_xpath('/html/body/pre')
        user=userID.get_attribute('innerHTML')
        s=user
        y=yaml.load(s)
        l=(y.get("logging_page_id"))
        IGid=(l.split('_'))
        IGidVal = IGid[1]
        driver.switch_to.window(driver.window_handles[0])
        if counter >= urlmax:
            print("ID Limit exceded")
            break
        return IGidVal

def myID():
    driver.switch_to.window(driver.window_handles[1])
    driver.get("https://www.instagram.com/"+login_id+"/?__a=1")
    time.sleep(0.3)
    userID=driver.find_element_by_xpath('/html/body/pre')
    user=userID.get_attribute('innerHTML')
    s=user
    y=yaml.load(s)
    l=(y.get("logging_page_id"))
    IGid=(l.split('_'))
    myIGidVal = IGid[1]
    driver.switch_to.window(driver.window_handles[0])
    return myIGidVal

def bot():
  scrollBot = 0 
  driver.get('https://www.instagram.com/')
  time.sleep(5)
  fBody = driver.find_element_by_xpath('//*[@id="react-root"]/section/main/section')
  for scroll in range(0,10):
    driver.execute_script("window.scrollTo(0, 1300)")
    time.sleep(5)
    # scrollBot += 1
    # if scroll == 10:
    #   print("Bot scrolling finish")
    #   break
    # else:
    #   pass'''

####################### THIS IS WHERE THE PROGRAM BEGINS #######################
infinity = 0x40000
maxCrawl = infinity
check = False
timeout = False
follGetError = False
timeoutValue = 10
timeoutCounter = 0
crawlCounter = 0
counter = 0
scroll = 0
folllist = []
follidlist = []
filename = "crawled_username.csv"

WINDOW_SIZE = "1360,768"
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument("--window-size=%s" % WINDOW_SIZE)
os.chdir("/content/gdrive/My Drive/")

## Deklarasikan driver sebagai ChromeDriver dengan cara seperti berikut:
driver = webdriver.Chrome(
    executable_path="/usr/lib/chromium-browser/chromedriver",
    chrome_options=chrome_options
)

## Setelah mounting ke akun Google Drive anda pada sel pertama, pastikan script berjalan di sana, dengan mengubah directory ke My Drive menggunakan fungsi os.chdir()
os.chdir("/content/gdrive/My Drive/")
## Buat directory baru di dalam My Drive yang bernama "instagram-crawling" untuk meletakkan output dari program ini. Membuat directory baru dengan menggunakan fungsi os.mkdir()
try:
  os.mkdir("./instagram-crawling")
except:
  pass
## Ubah directory ke dalam folder "instagram-crawling" yang baru saja dibuat
os.chdir("/content/gdrive/My Drive/instagram-crawling")
## Buat directory baru lagi yang bernama "instagram-usernames" untuk meletakkan semua username yang akan didapatkan dari program ini.
try:
    os.mkdir("./instagram-usernames")
except:
    pass
## Ubah lagi directory ke dalam folder "instagram-usernames" yang baru dibuat.
os.chdir("/content/gdrive/My Drive/instagram-crawling/instagram-usernames")

## Buka file "crawled_username.csv", yang berisi database dari username-username yang sudah dicrawl sebelumnya untuk mengecek apakah file tersebut sudah ada. Apabila belum ada, buat file csv baru yang bernama "crawled_username.csv"
try:
  open('crawled_username.csv', 'r')
  pass
except:
  with open('crawled_username.csv', 'w', newline='') as csvfile:
    wr = csv.writer(csvfile)

## Jalankan fungsi login_instagram() untuk login ke instagram dengan menggunakan username dan password yang telah dimasukkan pada sel ke tiga. 
## Login dilakukan agar bisa mengakses list followers yang ada pada profil user
login_instagram()
## Tambahkan waktu delay selama 5 detik untuk mensimulasikan interaksi orang pada saat masuk ke instagram.
time.sleep(5)
## Hitung berapa banyak username yang dimasukkan pada list usernames_target, pada sel keempat.
try:
  usernames_total = len(usernames_target)
except:
  usernames_target=[]
  usernames_total = len(usernames_target)
print("total of usernames to crawl: " + str(usernames_total))
## Jumlah username digunakan untuk mengetes apabila tidak ada username yang dimasukkan pada list di atas, maka anda akan diminta untuk memasukkannya disini.
if usernames_total == 0:
  sys.stderr.write("There is no username to crawl. Please enter at least 1 username to crawl! \n")
  usernames_target = input("Enter username to crawl here: ")
else:
  pass

## Looping mengambil data username dari list followers pada profil instagram dari username yang ada pada list usernames_target
while crawlCounter <= maxCrawl:
  ## Mengambil username yang ada pada usernames_target. Index yang menentukan username keberapa yang diambil diperoleh dari variabel "crawlCounter" yang dimulai dari angka 0 (pertama).
  try:  
    user_target = usernames_target[crawlCounter]
  except:
    sys.stderr.write("There is no username to crawl. The list is empty. \n")
    driver.close()
    sys.exit()
  
  ## Mengecek username yang ada di file "crawled_username.csv", apabila username sudah ada pada file tersebut, username tersebut tidak akan dicrawl lagi. Variabel "filename" harap dideklarasikan sebagai "crawled_username.csv" di awal program.
  with open(filename) as csvread:
    reader = csv.reader(csvread)
    for row in reader:
      if user_target in row:
        print("This username is already crawled")
        crawlCounter += 1
        check = True
      else:
        check = False
  if check == True:
    continue
  elif check == False:
    pass
  
  '''driver.execute_script("window.open('');")
  driver.switch_to.window(driver.window_handles[0])'''

  ## Masuk ke url dari profil instagram dari username yang akan dicrawl, untuk mengumpulkan data user.
  driver.get('https://www.instagram.com/' + user_target + '/')
  ## Variabel "crawlCounter" ditambahkan dengan angka 1 agar index yang diberikan untuk mengambil "usernames_target" berikutnya tepat.
  crawlCounter += 1
  ## Tambahkan waktu delay lagi selama 5 detik untuk mensimulasikan interaksi orang pada saat membuka profil instagram.
  time.sleep(5)

  '''timeoutCounter += 1
  if timeoutCounter == 3:
    timeoutCounter = 0
    timeout = True
    print("timeout engage")
  else:
    timeout = False
    pass'''

  ## Panggil fungsi followerButton() untuk mendapat nilai dari jumlah follower akun instagram yang dicrawl dan menekan tombol "followers" agar list follower keluar dan bisa diakses.
  try:
    follower_button=followerButton()
  except:
    sys.stderr.write("This account is private or doesn't exist \n")
    continue

  ## Jalankan userProfile() untuk mendapatkan nama lengkap, jumlah post, dan jumlah following dari username. 
  full_name, post_button, following_button = userProfile()
  ## Convert nilai dari jumlah folower yang ada pada variabel follower_button menjadi integer agar bisa dimasukkan menjadi argument yang menandakan jumlah total username pada saat mengambil username yang ada pada list follower.
  follower_value = int(follower_button)

  '''myIDstr = str(myID())
  print(myIDstr)'''
  
  ## Apabila username tersebut tidak memiliki follower, break untuk lanjut ke username berikutnya.
  if follower_value == 0:
    sys.stderr.write("This username has no follower \n")
    break
  else:
    pass
  ## Tambahkan waktu delay selama 3 detik untuk mensimulasikan interaksi orang pada saat membuka list follower.
  time.sleep(3)

  ## Scroll list follower hingga kurang lebih sesuai dengan limit maksimal username yang dicrawl yang ada pada fungsi follGet() (untuk 300 followers kira-kita 100x maximal scroll)
  fBody = driver.find_element_by_xpath("//div[@class='isgrP']")
  for scroll in range(0,100):
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollTop + arguments[0].offsetHeight;', fBody)
    time.sleep(0.2)
  print("scrolling finish")
  time.sleep(1)
  
  ## Loop crawling username dari list follower dengan parameter yang berupa urutan dari pertama hingga terakhir (argument follower_value)
  for i in range(1, follower_value):
    ## Jalankan follGet() dan masukkan "i" sebagai parameter dari fungsi tersebut. "i" akan berupa angka yang akan diproses oleh follGet() menjadi index untuk menentukan xpath dari username yang akan diambil.
    try:
      USERstr = str(follGet(i))
      follGetError = False
    ## If there is an error when collecting usernames from followers list, set follGetError to True and break
    except:
      sys.stderr.write("error occured \n")
      follGetError = True
      break
    print(str(i)+USERstr)
    if USERstr == "None":
      break
    else:
      pass
    '''try:
      IDstr = str(ID())
      follGetError = False
    except:
      sys.stderr.write("error occured \n")
      follGetError = True
      break
    print(IDstr)
    if IDstr == "None":
        IDstr = "ID limit exceded"
    else:
        pass
    follidlist.append(IDstr)'''
    ## Append hasil username yang baru saja diperoleh ke dalam list usernames_target, untuk kembali dicrawl lagi followernya. Lalu append hasil tersebut juga ke folllist untuk dimasukkan ke dalam file csv.
    usernames_target.append(USERstr)
    folllist.append(USERstr)

  if follGetError == False:
    print(folllist)
    folllistRow1 = folllist[0]
    folllist = folllist[1:follower_value]
  else:
    continue
  
  ## Append username yang sudah di crawled ke database "crawled_username.csv"
  with open('crawled_username.csv', 'a+', newline='') as csvwrite:
    wr = csv.writer(csvwrite)
    wr.writerow([user_target])

  ## Buat file csv baru yang nama filenya berdasarkan username yang dicrawl. Selain itu, sertai pula di dalamnya data-data lain yang tadi telah dikumpulkan.
  with open('followers(' + user_target + ').csv', 'w', newline='') as csvfile:
      wr = csv.writer(csvfile)
      wr.writerow(["Root"] + ["Username"] + ["Full Name"] + ["Total Posts"] + ["Total Followers"] + ["Total Following"] + ["Followers"])
      wr.writerow([login_id] + [user_target] + [full_name] + [post_button] + [follower_button] + [following_button] + [folllistRow1])
      for follower in folllist:
        wr.writerow([""]+[""]+[""]+[""]+[""]+[""]+[follower])
  
  ## Buka window baru untuk menghindari error saat crawling berikutnya
  driver.execute_script("window.open('');")
  ## Tutup window yang terbuka saat sesi crawl ini
  try:
    driver.switch_to.window(driver.window_handles[1])
    driver.execute_script("window.close('');")
  except:
    pass
  try:
    driver.switch_to.window(driver.window_handles[0])
    driver.execute_script("window.close('');")
  except:
    pass
  
  ## Bersihkan folllist dengan cara mereset listnya menjadi kosong
  folllist=[]
  ## Hapus username yang barusan dicrawl dari list usernames_target
  usernames_target.remove(user_target)

  '''time.sleep(timeoutValue)
  if timeout == True:
    print("restart")
    if __name__ == '__main__':
      main()
      os.execv(sys.executable, [sys.executable] + sys.argv)
  else:
    time.sleep(3)
  bot()'''

## Bersihkan usernames_target, untuk memastikan list tersebut kosong pada saat akan menjalankan program ini lagi. Lalu, tutup ChromeDriver nya.
usernames_target=[]
time.sleep(3)
driver.close()